In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
#import ESMF
import numpy as np
import pandas as pd
import xarray as xr
import pyroms

In [3]:
grd = pyroms.grid.get_ROMS_grid('SEAKp_1km')

Assuming spherical is integer 1 <class 'numpy.ma.core.MaskedArray'>
Load geographical grid from file


In [4]:
grd

In [5]:
lon2 = xr.DataArray(grd.hgrid.lon_psi)

In [6]:
ds_out = xr.Dataset({'lon': xr.DataArray(grd.hgrid.lon_psi), 'lat':xr.DataArray(grd.hgrid.lat_psi)})

In [7]:
ds_out

<xarray.Dataset>
Dimensions:  (dim_0: 281, dim_1: 325)
Dimensions without coordinates: dim_0, dim_1
Data variables:
    lon      (dim_0, dim_1) float64 212.0 212.1 212.1 ... 215.1 215.2 215.2
    lat      (dim_0, dim_1) float64 48.03 48.06 48.08 48.11 ... 65.17 65.2 65.22

In [8]:
river_data = xr.open_dataset('/import/AKWATERS/kshedstrom/hydroflow//new_2019/goa_dischargex_09012013_08312014.nc')

In [9]:
river_data

<xarray.Dataset>
Dimensions:  (time: 365, timeSeries: 14052)
Dimensions without coordinates: time, timeSeries
Data variables:
    q        (time, timeSeries) float32 ...
    lon      (timeSeries) float32 ...
    lat      (timeSeries) float32 ...
    year     (time) uint32 ...
    month    (time) uint32 ...
    day      (time) uint32 ...
Attributes:
    title:          Coastal Freshwater Discharge into the Gulf of Alaska
    summary:        Coastal FWD was modeled using a suite of physically based...
    keywords:       DISCHARGE/FLOW, ALASKA, GULF OF ALASKA
    date created:   Dataset created June 2019
    creator_name:   David Hill, Professor
    creator_email:  dfh@oregonstate.edu
    institution:    Oregon State University, Department of Civil Engineering

In [10]:
len(river_data.year)

365

In [11]:
river_data.lon[0:10]+360, river_data.lat[0:10]

(<xarray.DataArray 'lon' (timeSeries: 10)>
 array([205.27757, 205.26134, 205.29362, 205.26097, 205.3097 , 205.24455,
        205.32576, 205.22832, 205.24437, 205.34167], dtype=float32)
 Dimensions without coordinates: timeSeries,
 <xarray.DataArray 'lat' (timeSeries: 10)>
 array([56.418037, 56.41794 , 56.4271  , 56.43587 , 56.436157, 56.444733,
        56.445217, 56.44463 , 56.453697, 56.463234], dtype=float32)
 Dimensions without coordinates: timeSeries
 Attributes:
     long_name:  latitude
     units:      degrees_north)

In [12]:
len(river_data.timeSeries)

14052

In [13]:
lon2d = grd.hgrid.lon_psi
lat2d = grd.hgrid.lat_psi
Mm, Ll = lat2d.shape

In [14]:
ii, jj = np.meshgrid(range(Ll), range(Mm))
grd2 = list( zip(np.ravel(lon2d), np.ravel(lat2d)) )
idx = list( zip(np.ravel(ii), np.ravel(jj)) )

In [15]:
pts = list( zip(np.ravel(river_data.lon + 360.), np.ravel(river_data.lat)) )

In [16]:
from scipy import spatial
distance, index = spatial.KDTree(grd2).query(pts)

In [17]:
pts_coord = np.asarray(grd2)[index]
pts_idx = np.asarray(idx)[index]
pts_coord

array([[205.26645132,  56.41341473],
       [205.26645132,  56.41341473],
       [205.31316416,  56.44461114],
       ...,
       [210.62113403,  61.55675419],
       [210.68197962,  61.52861032],
       [210.68197962,  61.52861032]])

In [18]:
pts_idx

array([[ 84, 216],
       [ 84, 216],
       [ 85, 216],
       ...,
       [224, 250],
       [224, 249],
       [224, 249]])

In [19]:
indices = xr.Dataset({'i': (['river'], pts_idx[:,0]), 'j': (['river'], pts_idx[:,1])})

In [20]:
indices

<xarray.Dataset>
Dimensions:  (river: 14052)
Dimensions without coordinates: river
Data variables:
    i        (river) int64 84 84 85 84 85 85 85 ... 223 223 224 224 224 224 224
    j        (river) int64 216 216 216 216 216 217 ... 249 250 249 250 249 249

In [21]:
#indices.to_netcdf('indices.nc')

In [22]:
import pyroms_toolbox
width = 1
idx = []
idy = []
maskl = grd.hgrid.mask_rho.copy()
for w in range(width):
    lit = pyroms_toolbox.get_littoral2(maskl)
    idx.extend(lit[0])
    idy.extend(lit[1])
    maskl[lit] = 0

In [23]:
littoral_idx = (np.array(idx), np.array(idy))
maskl = np.zeros(grd.hgrid.mask_rho.shape)
maskl[littoral_idx] = 1

In [24]:
mask_idx = np.where(grd.hgrid.mask_rho == 0)

Make the runoff file

In [25]:
import netCDF4 as netCDF
from datetime import datetime
nt = len(river_data.time)
Mp, Lp = grd.hgrid.mask_rho.shape
#spval = -1e30
spval = -9999.
runoff_raw = np.zeros((Mp,Lp))
runoff = np.zeros((Mp,Lp))

# create runoff file
runoff_file = 'runoff_NGOA_Hill_2014.nc'
nc = netCDF.Dataset(runoff_file, 'w', format='NETCDF3_64BIT')
nc.Description = 'Hill & Beamer monthly climatology river discharge'
nc.Author = 'make_runoff_clim.py'
nc.Created = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
nc.title = 'Hill & Beamer river discharge'

# create dimensions and variables
nc.createDimension('xi_rho', np.size(grd.hgrid.mask_rho,1))
nc.createDimension('eta_rho', np.size(grd.hgrid.mask_rho,0))
nc.createDimension('runoff_time', None)

nc.createVariable('lon_rho', 'f8', ('eta_rho', 'xi_rho'))
nc.variables['lon_rho'].long_name = 'longitude of RHO-points'
nc.variables['lon_rho'].units = 'degree_east'
nc.variables['lon_rho'].field = 'lon_rho, scalar'
nc.variables['lon_rho'][:] = grd.hgrid.lon_rho

nc.createVariable('lat_rho', 'f8', ('eta_rho', 'xi_rho'))
nc.variables['lat_rho'].long_name = 'latitude of RHO-points'
nc.variables['lat_rho'].units = 'degree_north'
nc.variables['lat_rho'].field = 'lat_rho, scalar'
nc.variables['lat_rho'][:] = grd.hgrid.lat_rho

nc.createVariable('runoff_time', 'f8', ('runoff_time'))
nc.variables['runoff_time'].long_name = 'time'
nc.variables['runoff_time'].units = 'days since 1900-01-01 00:00:00'
#nc.variables['runoff_time'].cycle_length = 365.25

nc.createVariable('Runoff_raw', 'f8', ('runoff_time', 'eta_rho', 'xi_rho'), fill_value=spval)
nc.variables['Runoff_raw'].long_name = 'Dai_Trenberth River Runoff raw'
nc.variables['Runoff_raw'].units = 'kg/s/m^2'

nc.createVariable('Runoff', 'f8', ('runoff_time', 'eta_rho', 'xi_rho'), fill_value=spval)
nc.variables['Runoff'].long_name = 'Dai_Trenberth River Runoff'
nc.variables['Runoff'].units = 'kg/s/m^2'


In [26]:
import datetime
date1 = datetime.date(1900, 1, 1)

In [27]:
river_data.q.shape

(365, 14052)

In [28]:
nt

365

In [29]:
river_data.q

<xarray.DataArray 'q' (time: 365, timeSeries: 14052)>
[5128980 values with dtype=float32]
Dimensions without coordinates: time, timeSeries
Attributes:
    long_name:       Daily discharge
    units:           cubic meters per day
    coordinates:     time lat lon
    standard_name:   runoff_flux
    standard_units:  kg m-2 s-1

In [30]:
for t in range(nt):
    flow = np.sum(river_data.q[t,:])
    dater = datetime.date(river_data.year[t], river_data.month[t], river_data.day[t])
    time = (dater - date1).days
    print('Remapping runoff for time %f' %time)
    runoff_raw = np.zeros((Mp,Lp))
    runoff = np.zeros((Mp,Lp))
    for ri in range(len(river_data.timeSeries)):
        i = indices.i[ri]
        j = indices.j[ri]
        if (i > 323 or j == 0):
   #         print("overflow", i, j, ri)
            continue
        runoff_raw[j,i] += float(river_data.q[t,ri])
        
    nflow = np.sum(runoff_raw)
    print('time', t, flow/nflow)
    idx = np.where(runoff_raw != 0)
    runoff = pyroms_toolbox.move_runoff(runoff_raw, \
                  np.array(idx).T + 1, np.array(littoral_idx).T + 1, maskl, \
                  grd.hgrid.x_rho, grd.hgrid.y_rho, grd.hgrid.dx, grd.hgrid.dy)

# write data in destination file
    nc.variables['Runoff'][t] = runoff
    nc.variables['Runoff_raw'][t] = runoff_raw
    nc.variables['runoff_time'][t] = time

    if t==5:
        print('Sum 3', np.sum(runoff_raw))
        print('Sum 4', np.sum(runoff))

nc.close()

Remapping runoff for time 41516.000000
time 0 <xarray.DataArray 'q' ()>
array(1.05182417)
Remapping runoff for time 41517.000000
time 1 <xarray.DataArray 'q' ()>
array(1.05023929)
Remapping runoff for time 41518.000000
time 2 <xarray.DataArray 'q' ()>
array(1.04427626)
Remapping runoff for time 41519.000000
time 3 <xarray.DataArray 'q' ()>
array(1.03888008)
Remapping runoff for time 41520.000000
time 4 <xarray.DataArray 'q' ()>
array(1.03817788)
Remapping runoff for time 41521.000000
time 5 <xarray.DataArray 'q' ()>
array(1.03517353)
Sum 3 4309000674.090607
Sum 4 4308870190.151779
Remapping runoff for time 41522.000000
time 6 <xarray.DataArray 'q' ()>
array(1.02960124)
Remapping runoff for time 41523.000000
time 7 <xarray.DataArray 'q' ()>
array(1.02737524)
Remapping runoff for time 41524.000000
time 8 <xarray.DataArray 'q' ()>
array(1.027699)
Remapping runoff for time 41525.000000
time 9 <xarray.DataArray 'q' ()>
array(1.02828523)
Remapping runoff for time 41526.000000
time 10 <xarray

time 90 <xarray.DataArray 'q' ()>
array(1.37496241)
Remapping runoff for time 41607.000000
time 91 <xarray.DataArray 'q' ()>
array(1.38284797)
Remapping runoff for time 41608.000000
time 92 <xarray.DataArray 'q' ()>
array(1.35837637)
Remapping runoff for time 41609.000000
time 93 <xarray.DataArray 'q' ()>
array(1.31652743)
Remapping runoff for time 41610.000000
time 94 <xarray.DataArray 'q' ()>
array(1.27032639)
Remapping runoff for time 41611.000000
time 95 <xarray.DataArray 'q' ()>
array(1.22617212)
Remapping runoff for time 41612.000000
time 96 <xarray.DataArray 'q' ()>
array(1.18674792)
Remapping runoff for time 41613.000000
time 97 <xarray.DataArray 'q' ()>
array(1.15399867)
Remapping runoff for time 41614.000000
time 98 <xarray.DataArray 'q' ()>
array(1.13735254)
Remapping runoff for time 41615.000000
time 99 <xarray.DataArray 'q' ()>
array(1.13112322)
Remapping runoff for time 41616.000000
time 100 <xarray.DataArray 'q' ()>
array(1.1373797)
Remapping runoff for time 41617.000000

time 180 <xarray.DataArray 'q' ()>
array(1.09660496)
Remapping runoff for time 41697.000000
time 181 <xarray.DataArray 'q' ()>
array(1.10363362)
Remapping runoff for time 41698.000000
time 182 <xarray.DataArray 'q' ()>
array(1.10694723)
Remapping runoff for time 41699.000000
time 183 <xarray.DataArray 'q' ()>
array(1.10884278)
Remapping runoff for time 41700.000000
time 184 <xarray.DataArray 'q' ()>
array(1.11184927)
Remapping runoff for time 41701.000000
time 185 <xarray.DataArray 'q' ()>
array(1.11346991)
Remapping runoff for time 41702.000000
time 186 <xarray.DataArray 'q' ()>
array(1.11516738)
Remapping runoff for time 41703.000000
time 187 <xarray.DataArray 'q' ()>
array(1.20194517)
Remapping runoff for time 41704.000000
time 188 <xarray.DataArray 'q' ()>
array(1.3119194)
Remapping runoff for time 41705.000000
time 189 <xarray.DataArray 'q' ()>
array(1.32535385)
Remapping runoff for time 41706.000000
time 190 <xarray.DataArray 'q' ()>
array(1.29476597)
Remapping runoff for time 41

time 270 <xarray.DataArray 'q' ()>
array(1.16753672)
Remapping runoff for time 41787.000000
time 271 <xarray.DataArray 'q' ()>
array(1.15560819)
Remapping runoff for time 41788.000000
time 272 <xarray.DataArray 'q' ()>
array(1.14280829)
Remapping runoff for time 41789.000000
time 273 <xarray.DataArray 'q' ()>
array(1.13820092)
Remapping runoff for time 41790.000000
time 274 <xarray.DataArray 'q' ()>
array(1.14020673)
Remapping runoff for time 41791.000000
time 275 <xarray.DataArray 'q' ()>
array(1.14334707)
Remapping runoff for time 41792.000000
time 276 <xarray.DataArray 'q' ()>
array(1.1464557)
Remapping runoff for time 41793.000000
time 277 <xarray.DataArray 'q' ()>
array(1.1487191)
Remapping runoff for time 41794.000000
time 278 <xarray.DataArray 'q' ()>
array(1.14872528)
Remapping runoff for time 41795.000000
time 279 <xarray.DataArray 'q' ()>
array(1.14580073)
Remapping runoff for time 41796.000000
time 280 <xarray.DataArray 'q' ()>
array(1.14270594)
Remapping runoff for time 417

time 360 <xarray.DataArray 'q' ()>
array(1.05912947)
Remapping runoff for time 41877.000000
time 361 <xarray.DataArray 'q' ()>
array(1.0575931)
Remapping runoff for time 41878.000000
time 362 <xarray.DataArray 'q' ()>
array(1.0618297)
Remapping runoff for time 41879.000000
time 363 <xarray.DataArray 'q' ()>
array(1.06914706)
Remapping runoff for time 41880.000000
time 364 <xarray.DataArray 'q' ()>
array(1.08008935)
